# Getting user rating by hybrid filtering

In [8]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr

In [9]:
"""similarity measure"""
def sim(x, y, metric='cos'):
    if metric == 'cos':
        return 1.-cosine(x,y)
    else:
        return pearsonr(x,y)[0]

In [ ]:
ufeat_matrix = pd.read_csv('added_sparse_matrix.csv')

# Making similarity matrix

In [16]:
class Hybrid_cbf_cf(object):
    def __init__(self, ufeat_matrix):
        #initial value
        self.ufeat_matrix = ufeat_matrix
        self.nusers = len(ufeat_matrix)
        self.nani = len(ufeat_matrix[0])
    
    
    def CalcRatings(self,u_vec_feats,K):
        data_sim = np.zeros((self.nusers, self.nani + 1))  # Array for ufeat_matrix and similarity """
        data_sim[:,:-1] = self.ufeat_matrix
        u_rec = np.zeros(len(ufeat_matrix[0,:-43]))  #Array for rating animation for each user 'u' """
        for u in range(self.nusers):
            if np.array_equal(data_sim[u,:-1], u_vec_feats) == False:
                data_sim[u, self.nani] = sim(data_sim[u,:-1], u_vec_feats)
            else:
                data_sim[u,ncols] = 0
                
        data_sim = data_sim[data_sim[:,ncols].argsort()][::-1] # Ordered by similarity

        def FindKNeighbours(r,data_sim,K):
            neighs = []
            cnt=0
            for u in range(len(data_sim)):
                if data_sim[u,r]>0 and cnt<K: # Finding K neighbors for item 'r'
                   neighs.append(data_sim[u])   
                   cnt +=1 
                elif cnt==K:
                   break
            return neighs
        
        def CalcRating(u_vec_feats,r,neighs):
            rating = 0.
            den = 0.
            for j in range(len(neighs)):
                 # Multipling similarity by centered value of user 'j'th item 'r' and summing the values
                rating += neighs[j][-1]*float(neighs[j][r]-neighs[j][neighs[j]>0][:-1].mean())
                den += abs(neighs[j][-1]) # Absolute value of similarity
            if den>0:
                rating = u_vec_feats[:self.nani][u_vec_feats[:self.numatrix]>0].mean()+(rating/den)
            else:
                rating = u_vec_feats[:self.nani][u_vec_feats[:self.numatrix]>0].mean()
            if rating>10:
                return 10. # The maximum value
            elif rating<0:
                return 0. # The minimum value
            return rating
        
        for r in range(self.nani):
            if u_vec_feats[r]==0:
               neighs = FindKNeighbours(r,data_sim,K)
               #calc the predicted rating
               u_rec[r] = CalcRating(u_vec_feats,r,neighs)
        return u_rec